# tool_merge_encoded_messages_all_senders

# Main

In [1]:
import dfBasics
import common
import encoder
import pfAdapt
#import charts

In [2]:
version_sla = 'v00001'
version     = version_sla + '/v00000'

home_directory  =  '/home/jovyan/work/'
share_directory =  '/home/jovyan/work/share/'
#share_directory =  '/home/jovyan/share/'

In [3]:
import pandas as pd
from pyspark.sql import functions

In [4]:
columns = ['CGLOBALMESSAGEID', 'CSTARTTIME', 'CENDTIME', 'CSTATUS', 'CSERVICE',\
       'CSLABILLINGMONTH', 'CSENDERPROTOCOL', 'CSENDERENDPOINTID',\
       'CINBOUNDSIZE', 'CRECEIVERPROTOCOL', 'CRECEIVERENDPOINTID', 'CSLATAT',\
       'CMESSAGETAT2', 'CSLADELIVERYTIME']
# withot 'CSLABILLINGMONTH'
def get_columns_2():
    columns = ['CGLOBALMESSAGEID', 'CSTARTTIME', 'CENDTIME', 'CSTATUS', 'CSERVICE',\
            'CSENDERPROTOCOL', 'CSENDERENDPOINTID',\
           'CINBOUNDSIZE', 'CRECEIVERPROTOCOL', 'CRECEIVERENDPOINTID', 'CSLATAT',\
           'CMESSAGETAT2', 'CSLADELIVERYTIME']
    return columns

columns = ['CGLOBALMESSAGEID',  'CSTARTTIME', 'CENDTIME', 'CSTATUS', 'CSERVICE', 'CSENDERENDPOINTID', 'CSENDERPROTOCOL', 'CINBOUNDSIZE', 'CRECEIVERPROTOCOL', 'CRECEIVERENDPOINTID', 'CSLATAT', 'CMESSAGETAT2', 'CSLADELIVERYTIME']
     

#columns = get_columns_2()
#to count messages sent
#columns = [ 'CSTARTTIME', 'CSENDERENDPOINTID']

In [5]:
sparkSession = dfBasics.getSparkSession()

In [6]:
df = sparkSession.read.parquet('hdfs://172.30.17.145:8020/sla_sql_data/*/*').select(columns).dropDuplicates() 

In [7]:
#senders = sparkSession.read.parquet("/tmp/senders.parquet")
#senders = pd.read_parquet('/tmp/senders' + '.parquet', engine='pyarrow')
#senders = list(senders.toPandas()['CSENDERENDPOINTID'])

In [8]:
senders = list(sparkSession.read.parquet('hdfs://172.30.17.145:8020/user/admin/sla/v00000/v00000/senders/senders.parquet').toPandas()['CSENDERENDPOINTID'])

In [9]:
len(senders)

5493

In [10]:
#senders

In [11]:
senders_files = sparkSession.read.options(delimiter=',') \
                      .csv('hdfs://172.30.17.145:8020/user/admin/sla/' + version + '/senders/senders_files.txt').toPandas()
senders_files.columns = ['size','filename']    

In [12]:
#senders_files['filename'][0]

In [13]:
#'sla_enc_' + senders[0] + '.parquet'

In [14]:
failed = []
for sender in senders:
    try:
        filename = 'sla_enc_' + sender + '.parquet'
        sparkSession.read.text('hdfs://172.30.17.145:8020/user/admin/sla/' + version + '/encoded/senders/' + filename + '/_SUCCESS')
    except Exception as exception: 
        failed.append(sender)
        #if not 'temporary' in filename:
            #failed.append(filename.split('sla_enc_')[1].split('.')[0])

In [15]:
failed.remove(None)
len(failed)

2050

In [16]:
working = [x for x in senders if x not in failed]
working.remove(None)

In [17]:
len(working)

3442

In [18]:
#sender = working[0]
#filename = 'sla_enc_' + sender + '.parquet'
#_df = sparkSession.read.parquet('hdfs://172.30.17.145:8020/user/admin/sla/' + version + '/encoded/senders/' + filename ).toPandas()
#_df.columns

In [19]:
columns = ['CGLOBALMESSAGEID', 'CSTARTTIME', 'CENDTIME', 'CSTATUS', 'CSERVICE', 'CSENDERENDPOINTID', 'CSENDERPROTOCOL', 'CINBOUNDSIZE', 'CRECEIVERPROTOCOL', 'CRECEIVERENDPOINTID', 'CSLATAT', 'CMESSAGETAT2', 'CSLADELIVERYTIME', 'year', 'month', 'day', 'hour', 'minute']

In [20]:
#columns

In [21]:
df = pd.DataFrame(columns=columns)
#df.append(_df)
#df

In [ ]:
for sender in working:
    #print(sender)
    try:
        filename = 'sla_enc_' + sender + '.parquet'
        _df = sparkSession.read.parquet('hdfs://172.30.17.145:8020/user/admin/sla/' + version + '/encoded/senders/' + filename ).toPandas()
        df = df.append(_df)
    except Exception as exception:
        pass

In [ ]:
#!ls -l /home/jovyan/work/output/sla_enc_all.parquet

In [23]:
df.to_parquet('/home/jovyan/work/output/sla_enc_all_4.parquet')

In [ ]:
#pd.read_parquet('/home/jovyan/work/output/sla_enc_all_2.parquet')

In [24]:
#_senders = pd.unique(df['CSENDERENDPOINTID'])
#len(_senders)

975

In [ ]:
#pd.unique(df[df['CSENDERENDPOINTID'] == _senders[0]]['CRECEIVERPROTOCOL'])

In [ ]:
#pd.unique(df['CRECEIVERPROTOCOL'])